In [ ]:
# If we don't have the data, download it
!wget https://dmlab.cs.gsu.edu/solar/data/data-comp-2020/train_partition1_data.json
!mkdir data
!mv train_partition1_data.json data/

In [3]:
import pandas as pd
import numpy as np
import torch
import json
from normalizer import counter, subSample
from torch.utils.data import DataLoader
from normalizer import subSample, getDataFromJSON

device = 'cuda' if torch.cuda.is_available() else 'cpu' # Change this to cuda for GPU enabled computers
device

'cpu'

In [4]:
train1, labels1 = subSample("data/train_partition1_data.json", device=device)

Now reading data/train_partition1_data.json
Now loading event 1/785
Now loading event 101/785
Now loading event 201/785
Now loading event 301/785
Now loading event 401/785
Now loading event 501/785
Now loading event 601/785
Now loading event 701/785
785 lines loaded.


In [5]:
for X in [train1]: # This line edited for easier loading
    for i in range(X.shape[1]): # the number of types of measurements
        min_x = torch.min(X[:,i,:])
        X[:,i,:] -= min_x
        X[:,i,:] /= torch.max(X[:,i,:]).item()

In [6]:
# Define the network. Make sure to end with nn.Softmax activation
import torch.nn as nn
# from skorch import NeuralNet

class logRegWithHidden(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, num_classes=5, drop1=.5, input_size=1980):
        super().__init__() 
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layerout = nn.Linear(hidden_size2, num_classes)
        #Define a RELU Activation unit
        self.relu = nn.ReLU()  
        self.smax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(p=drop1)

    def forward(self, x):
        #Forward Propagate through the layers as defined above
        y = self.drop(x.reshape(-1, 1980))
        y = self.drop(self.relu(self.layer1(y)))
        y = self.relu(self.layer2(y))
        y = self.layerout(y)
        y = self.smax(y)
        return y



In [8]:
def train(modelModule, inputs, labels, weight, valSets, valLabels, valweight, *modelArgs, lr=0.01, **modelKwargs):
    
    # modify these variables to control how much is printed
    pmodel = False
    pstateDict = False
    pindvLoss = False
    pmodelDict = False
    pvalOut = False
    
    
    #TODO: does this work?
    model = modelModule(*modelArgs, **modelKwargs)
    if pmodel: print(model)
    
    if weight is not None: weight = torch.Tensor(weight)
    lfc = nn.CrossEntropyLoss(weight=weight)
#     valLabels = torch.tensor(valLabels, dtype=torch.int)
    #ideas
    # 1-(weight/np.sum(weight))
    # .2/weight - this one normalizes so that each class is responsible for 20% of the loss
    # 1/weight - this is a bit naive, but the classes with fewer items are weighted more.
    # 1/(weight+1) - makes sure we don't have any pesky zeroes
    # np.sum(weight)/weight if your learning rate is too low.
    
    # Hyperparameters
    batch = 256
    epochs = 4 # artificially low for debugging
    
    # Start a dataloader object
    data = list(zip(inputs,labels))
    val = list(zip(valSets,valLabels))
    loader = DataLoader(data, batch_size = batch, num_workers=4)
    valLoader = DataLoader(val, batch_size = int(len(val)/4), num_workers=4)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    if pstateDict: print(opt.state_dict())

    for epoch in range(epochs):
        model.train()
        batch_loss = []
        for (xtrain,ytrain) in loader:
            opt.zero_grad()
            if pstateDict: print(opt.state_dict())
            output = model(xtrain)
            loss = lfc(output,ytrain)
            if pindvLoss: print(loss)
            loss.backward()
            if pstateDict: print(opt.state_dict())
            if pmodelDict: print(model.state_dict())
            opt.step()
            if pstateDict: print(opt.state_dict())
            if pstateDict or pmodelDict: print('\n\n\n\n\n')
            batch_loss.append(loss.item())
        print(f'The training loss for epoch {epoch+1}/{epochs} was {np.mean(batch_loss)}')
        
        
        model.eval()
        balanced = [[],[],[],[],[]]
        batchLoss = []
        unbalanced = []
        
        for (xval,yval) in valLoader:
            output = model(xval)
            loss = lfc(output,yval)
            batchLoss.append(loss.item())
            if pvalOut : print(output)
            corrects = yval.clone().detach() == torch.argmax(output,1)
            if pvalOut: print(yval.clone().detach(), torch.argmax(output,1))
            if pvalOut: print(corrects.detach())
            if pvalOut: print('===========================\n\n\n')
            unbalanced.append(np.mean([1 if correct else 0 for correct in corrects.detach()]))
            
            for i, ans in enumerate(yval):
                balanced[ans].append(corrects[i])
        
        balanced = [np.mean(i) for i in balanced]
        balancedAccuracy = np.mean(balanced)
        
#         print(f'The total balanced accuracy for validation was {balancedAccuracy}')
        print(f'The values of unbalanced are {unbalanced}')
        print(f'The validation loss was :   {epoch+1}/{epochs} was {np.mean(batchLoss)}')
        print(f'The unbalanced validation accuracy is {np.mean(unbalanced)}')
        print(f'The accuracy for each is {balanced}')           
    print(opt.state_dict())


    return model


In [10]:
# [weights1[i] + weights2[i] for i in range(5)]
trainArgs = [logRegWithHidden, train1, labels1, None, train1, labels1, None]
modelArgs = [4096*4, 2048*4]
%time newModel = train(*trainArgs, *modelArgs, lr = 0.01, drop1=0)


The training loss for epoch 1/4 was 1.6757709085941315
The values of unbalanced are [0.20408163265306123, 0.25510204081632654, 0.16326530612244897, 0.17857142857142858, 0.0]
The validation loss was :   1/4 was 1.7446301937103272
The unbalanced validation accuracy is 0.16020408163265307
The accuracy for each is [0.0, 0.0, 0.0, 0.0, 1.0]
The training loss for epoch 2/4 was 1.7240572571754456
The values of unbalanced are [0.20408163265306123, 0.25510204081632654, 0.16326530612244897, 0.17857142857142858, 0.0]
The validation loss was :   2/4 was 1.7446301937103272
The unbalanced validation accuracy is 0.16020408163265307
The accuracy for each is [0.0, 0.0, 0.0, 0.0, 1.0]
The training loss for epoch 3/4 was 1.7240572571754456
The values of unbalanced are [0.20408163265306123, 0.25510204081632654, 0.16326530612244897, 0.17857142857142858, 0.0]
The validation loss was :   3/4 was 1.7446301937103272
The unbalanced validation accuracy is 0.16020408163265307
The accuracy for each is [0.0, 0.0, 0